In [ ]:
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # specify the path of the folder containing "file_name" :
    path_to_file = '/content/gdrive/My Drive/5153 gp/'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/5153 gp/
/content/gdrive/My Drive/5153 gp


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('movie_variable_full.csv')
df.head()

,id,tconst,titleType,primaryTitle,originalTitle,startYear,release_date,isAdult,genres,runtime,budget,original_language,overview,popularity,poster_path,production_companies,production_countries,revenue,status,vote_average,vote_count,averageRating,numVotes,director,actor
0,16612,tt0000001,short,Carmencita,Carmencita,1894.0,1894-03-14,0,"Documentary,Short",1.0,0,en,The first woman to appear in front of an Ediso...,1.273072,/6QJowxF4f2hrUtmrDXOqbRDMdDH.jpg,Edison Manufacturing Company,US,0.0,Released,4.9,18.0,5.6,1550.0,William K.L. Dickson,Carmencita
1,88013,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,1892.0,1892-10-28,0,"Animation,Comedy,Romance",4.0,0,fr,"One night, Arlequin come to see his lover Colo...",0.673164,/yCJr2zZqEbWLggyqumVcXBFQABf.jpg,None,FR,0.0,Released,6.1,19.0,6.5,1207.0,Émile Reynaud,None
2,16624,tt0000005,short,Blacksmith Scene,Blacksmith Scene,1893.0,1893-05-08,0,"Comedy,Short",1.0,0,xx,Three men hammer on an anvil and pass a bottle...,1.061591,/6nh4gdZ4cWQbnXDicWIGsOZiZsx.jpg,Edison Manufacturing Company,US,0.0,Released,5.8,19.0,6.1,1934.0,William K.L. Dickson,Charles Kayser
3,105158,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,1894.0,1894-01-09,0,"Documentary,Short",1.0,0,en,A man (Thomas Edison's assistant) takes a pinc...,0.312246,/utsYe7NaVdiUhXpBwgln3LTucYs.jpg,Edison Manufacturing Company,US,0.0,Released,4.7,12.0,5.4,1667.0,William K.L. Dickson,Fred Ott
4,774,tt0000010,short,Exiting the Factory,La sortie de l'usine Lumière à Lyon,1895.0,1895-06-10,0,"Documentary,Short",1.0,0,fr,Working men and women leave the Lumière factor...,0.693917,/zwaMUm6cxmbMIUziIhkqyIoi2LO.jpg,Société Lumière,FR,0.0,Released,6.2,52.0,6.9,5545.0,Louis Lumière,None


In [ ]:
df.shape

(45307, 25)

In [ ]:
df.columns

Index(['id', 'tconst', 'titleType', 'primaryTitle', 'originalTitle',
       'startYear', 'release_date', 'isAdult', 'genres', 'runtime', 'budget',
       'original_language', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'revenue', 'status',
       'vote_average', 'vote_count', 'averageRating', 'numVotes', 'director',
       'actor'],
      dtype='object')

In [ ]:
df.describe()

,id,startYear,isAdult,runtime,budget,popularity,revenue,vote_average,vote_count,averageRating,numVotes
count,45307.000000,45307.000000,45307.000000,45307.000000,4.530700e+04,45307.000000,4.530700e+04,45307.000000,45307.000000,45307.000000,4.530700e+04
mean,107996.021983,1991.857748,0.000596,98.097115,4.235149e+06,2.927732,1.124325e+07,5.625003,110.222151,6.322242,1.613350e+04
std,112202.464454,24.002295,0.024405,35.340389,1.745001e+07,6.013227,6.443678e+07,1.914672,492.101919,1.144720,6.829993e+04
min,2.000000,1874.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.100000,5.000000e+00
25%,26380.500000,1978.000000,0.000000,87.000000,0.000000e+00,0.388835,0.000000e+00,5.000000,3.000000,5.700000,5.050000e+02
50%,59828.000000,2001.000000,0.000000,96.000000,0.000000e+00,1.130550,0.000000e+00,6.000000,10.000000,6.500000,1.470000e+03
75%,156332.000000,2010.000000,0.000000,108.000000,0.000000e+00,3.694370,0.000000e+00,6.800000,34.000000,7.100000,5.356000e+03
max,469172.000000,2019.000000,1.000000,1256.000000,3.800000e+08,547.488298,2.787965e+09,10.000000,14075.000000,9.500000,2.159745e+06


In [ ]:
data = df[['titleType', 'startYear', 'release_date', 'isAdult', 'genres', 'runtime', 'budget',
       'original_language', 'popularity', 'production_companies', 'production_countries', 'revenue', 'status',
       'vote_average', 'vote_count', 'numVotes', 'director', 'actor', 'averageRating']].copy()

In [ ]:
data['titleType'].value_counts()

movie           39077
tvMovie          2570
short            1326
video            1323
tvSpecial         387
tvMiniSeries      327
tvEpisode         152
tvShort            93
tvSeries           52
Name: titleType, dtype: int64

In [ ]:
data['titleType'] = pd.Categorical(data['titleType'])
data['titleType'] = data['titleType'].cat.codes

In [ ]:
data['titleType'].value_counts()

0    39077
4     2570
1     1326
8     1323
7      387
3      327
2      152
6       93
5       52
Name: titleType, dtype: int64

In [ ]:
# df_titletype = pd.get_dummies(data.titleType, prefix = 'type')
# data = pd.concat([data,df_titletype],axis = 1)
# data = data.drop('titleType',axis = 1)

In [ ]:
data.startYear = data.startYear.astype(int)
data['startYear'] = np.where(data['startYear'] < 2000,0,data['startYear'])
data['startYear'] = np.where((data['startYear'] >= 2000) & (data['startYear'] < 2010), 1, data['startYear'])
data['startYear'] = np.where(data['startYear'] >= 2010,2,data['startYear'])
data['startYear'].value_counts()

0    21369
2    12691
1    11247
Name: startYear, dtype: int64

In [ ]:
data.release_date = pd.to_datetime(data.release_date)
data['release_month'] = data.release_date.dt.month
data['release_month'] = data['release_month'].astype(int)
data = data.drop('release_date',axis = 1)

In [ ]:
genres_l = ['Action', 'Adult', 'Adventure', 'Animation', 
            'Biography', 'Comedy', 'Crime', 'Documentary', 
            'Drama', 'Family', 'Fantasy', 'Film-Noir', 
            'Foreign', 'History', 'Horror', 'Music', 'Musical', 
            'Mystery', 'News', 'Reality-TV', 'Romance', 
            'Sci-Fi', 'Short', 'Sport', 'TV Movie', 'Talk-Show', 
            'Thriller', 'War', 'Western']

df_genres = []
for i in data.genres:
    dic = dict()
    for j in i.split(','):
        dic[j] = 1
    df_genres.append(dic)

df_genres = pd.DataFrame(df_genres)
df_genres = df_genres[genres_l]
df_genres = df_genres.fillna(0)
df_genres = df_genres.astype(int)
df_genres.columns = ['genres_' + i for i in genres_l]

data = pd.concat([data,df_genres],axis = 1)
data = data.drop('genres',axis = 1)

In [ ]:
def top_rank_category(d, x, rank, name):
    L = list(d[x].value_counts().keys())
    if 'None' in L:
        L.remove('None')
    L = L[:rank] + ['others']

    df_ = []
    for i in d[x]:
        dic = dict()
        if i in L:
            dic[i] = 1
        else:
            dic['others'] = 1
        df_.append(dic)

    df_ = pd.DataFrame(df_)
    df_ = df_.fillna(0)
    df_ = df_.astype(int)
    df_ = df_[L]
    df_.columns = [name + i for i in L]
    
    return df_

df_temp = top_rank_category(data, 'original_language', 8, 'language_')
data = pd.concat([data,df_temp],axis = 1)
data = data.drop('original_language',axis = 1)

df_temp = top_rank_category(data, 'production_companies', 5, 'company_')
data = pd.concat([data,df_temp],axis = 1)
data = data.drop('production_companies',axis = 1)

df_temp = top_rank_category(data, 'production_countries', 6, 'country_')
data = pd.concat([data,df_temp],axis = 1)
data = data.drop('production_countries',axis = 1)

df_temp = top_rank_category(data, 'director', 10, 'director_')
data = pd.concat([data,df_temp],axis = 1)
data = data.drop('director',axis = 1)

df_temp = top_rank_category(data, 'actor', 10, 'actor_')
data = pd.concat([data,df_temp],axis = 1)
data = data.drop('actor',axis = 1)

In [ ]:
#  'language_en', 'language_fr', 'language_it', 'language_ja',
# 'language_de', 'language_es', 'language_ru', 'language_hi', 'language_others'
data['original_language'] = np.where(data['language_en']==1, 0, 1)
data['original_language'] = np.where(data['language_fr']==1, 1, data['original_language'])
data['original_language'] = np.where(data['language_it']==1, 2, data['original_language'])
data['original_language'] = np.where(data['language_ja']==1, 3, data['original_language'])
data['original_language'] = np.where(data['language_de']==1, 4, data['original_language'])
data['original_language'] = np.where(data['language_es']==1, 5, data['original_language'])
data['original_language'] = np.where(data['language_ru']==1, 6, data['original_language'])
data['original_language'] = np.where(data['language_hi']==1, 7, data['original_language'])
data['original_language'] = np.where(data['language_others']==1, 8, data['original_language'])
data = data.drop(columns=['language_en', 'language_fr', 'language_it', 'language_ja',
       'language_de', 'language_es', 'language_ru', 'language_hi',
       'language_others'])

In [ ]:
# 'company_Paramount Pictures', 'company_Metro-Goldwyn-Mayer (MGM)',
# 'company_Twentieth Century Fox Film Corporation',
# 'company_Warner Bros.', 'company_Universal Pictures', 'company_others'
data['production_companies'] = np.where(data['company_Paramount Pictures']==1, 0, 1)
data['production_companies'] = np.where(data['company_Metro-Goldwyn-Mayer (MGM)']==1, 1, data['production_companies'])
data['production_companies'] = np.where(data['company_Twentieth Century Fox Film Corporation']==1, 2, data['production_companies'])
data['production_companies'] = np.where(data['company_Warner Bros.']==1, 3, data['production_companies'])
data['production_companies'] = np.where(data['company_Universal Pictures']==1, 4, data['production_companies'])
data['production_companies'] = np.where(data['company_others']==1, 5, data['production_companies'])
data = data.drop(columns=['company_Paramount Pictures', 'company_Metro-Goldwyn-Mayer (MGM)',
       'company_Twentieth Century Fox Film Corporation',
       'company_Warner Bros.', 'company_Universal Pictures', 'company_others'])

In [ ]:
# production_countries
# 'country_US', 'country_GB', 'country_FR', 'country_CA', 'country_JP', 'country_IT', 'country_others'
data['production_countries'] = np.where(data['country_US']==1, 0, 1)
data['production_countries'] = np.where(data['country_GB']==1, 1, data['production_countries'])
data['production_countries'] = np.where(data['country_FR']==1, 2, data['production_countries'])
data['production_countries'] = np.where(data['country_CA']==1, 3, data['production_countries'])
data['production_countries'] = np.where(data['country_JP']==1, 4, data['production_countries'])
data['production_countries'] = np.where(data['country_IT']==1, 5, data['production_countries'])
data['production_countries'] = np.where(data['country_others']==1, 6, data['production_countries'])
data = data.drop(columns=['country_US', 'country_GB', 'country_FR', 'country_CA', 'country_JP',
       'country_IT', 'country_others'])

In [ ]:
# 'director_John Ford', 'director_Michael Curtiz',
# 'director_Werner Herzog', 'director_Alfred Hitchcock',
# 'director_Woody Allen', 'director_Georges Méliès',
# 'director_Sidney Lumet', 'director_Jean-Luc Godard',
# 'director_Charlie Chaplin', 'director_Raoul Walsh', 'director_others',
data['director'] = np.where(data['director_John Ford']==1, 0, 1)
data['director'] = np.where(data['director_Michael Curtiz']==1, 1, data['director'])
data['director'] = np.where(data['director_Werner Herzog']==1, 2, data['director'])
data['director'] = np.where(data['director_Alfred Hitchcock']==1, 3, data['director'])
data['director'] = np.where(data['director_Woody Allen']==1, 4, data['director'])
data['director'] = np.where(data['director_Georges Méliès']==1, 5, data['director'])
data['director'] = np.where(data['director_Sidney Lumet']==1, 6, data['director'])
data['director'] = np.where(data['director_Jean-Luc Godard']==1, 7, data['director'])
data['director'] = np.where(data['director_Charlie Chaplin']==1, 8, data['director'])
data['director'] = np.where(data['director_Raoul Walsh']==1, 9, data['director'])
data['director'] = np.where(data['director_others']==1, 10, data['director'])
data = data.drop(columns=['director_John Ford', 'director_Michael Curtiz',
       'director_Werner Herzog', 'director_Alfred Hitchcock',
       'director_Woody Allen', 'director_Georges Méliès',
       'director_Sidney Lumet', 'director_Jean-Luc Godard',
       'director_Charlie Chaplin', 'director_Raoul Walsh', 'director_others'])

In [ ]:
# 'actor_John Wayne', 'actor_Jackie Chan', 'actor_Nicolas Cage',
# 'actor_Robert De Niro', 'actor_Gérard Depardieu', 'actor_Michael Caine',
# 'actor_Burt Lancaster', 'actor_Paul Newman', 'actor_Bruce Willis',
# 'actor_Barbara Stanwyck', 'actor_others'
data['actor'] = np.where(data['actor_John Wayne']==1, 0, 1)
data['actor'] = np.where(data['actor_Jackie Chan']==1, 1, data['actor'])
data['actor'] = np.where(data['actor_Nicolas Cage']==1, 2, data['actor'])
data['actor'] = np.where(data['actor_Robert De Niro']==1, 3, data['actor'])
data['actor'] = np.where(data['actor_Gérard Depardieu']==1, 4, data['actor'])
data['actor'] = np.where(data['actor_Michael Caine']==1, 5, data['actor'])
data['actor'] = np.where(data['actor_Burt Lancaster']==1, 6, data['actor'])
data['actor'] = np.where(data['actor_Paul Newman']==1, 7, data['actor'])
data['actor'] = np.where(data['actor_Bruce Willis']==1, 8, data['actor'])
data['actor'] = np.where(data['actor_Barbara Stanwyck']==1, 9, data['actor'])
data['actor'] = np.where(data['actor_others']==1, 10, data['actor'])
data = data.drop(columns=['actor_John Wayne', 'actor_Jackie Chan', 'actor_Nicolas Cage',
       'actor_Robert De Niro', 'actor_Gérard Depardieu', 'actor_Michael Caine',
       'actor_Burt Lancaster', 'actor_Paul Newman', 'actor_Bruce Willis',
       'actor_Barbara Stanwyck', 'actor_others'])

In [ ]:
data.columns

Index(['titleType', 'startYear', 'isAdult', 'runtime', 'budget', 'popularity',
       'revenue', 'status', 'vote_average', 'vote_count', 'numVotes',
       'averageRating', 'release_month', 'genres_Action', 'genres_Adult',
       'genres_Adventure', 'genres_Animation', 'genres_Biography',
       'genres_Comedy', 'genres_Crime', 'genres_Documentary', 'genres_Drama',
       'genres_Family', 'genres_Fantasy', 'genres_Film-Noir', 'genres_Foreign',
       'genres_History', 'genres_Horror', 'genres_Music', 'genres_Musical',
       'genres_Mystery', 'genres_News', 'genres_Reality-TV', 'genres_Romance',
       'genres_Sci-Fi', 'genres_Short', 'genres_Sport', 'genres_TV Movie',
       'genres_Talk-Show', 'genres_Thriller', 'genres_War', 'genres_Western',
       'original_language', 'production_companies', 'production_countries',
       'director', 'actor'],
      dtype='object')

In [ ]:
data.status.value_counts()

Released           44876
Rumored              226
Post Production       95
None                  80
In Production         17
Planned               12
Canceled               1
Name: status, dtype: int64

In [ ]:
data['status'] = pd.Categorical(data['status'])
data['status'] = data['status'].cat.codes
data.status.value_counts()

5    44876
6      226
4       95
2       80
1       17
3       12
0        1
Name: status, dtype: int64

In [ ]:
# df_status = pd.get_dummies(data.status, prefix = 'status')
# data = pd.concat([data,df_status],axis = 1)
# data = data.drop(['status','status_None'],axis = 1)

In [ ]:
data.head(1)

,titleType,startYear,isAdult,runtime,budget,popularity,revenue,status,vote_average,vote_count,numVotes,averageRating,release_month,genres_Action,genres_Adult,genres_Adventure,genres_Animation,genres_Biography,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Film-Noir,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Musical,genres_Mystery,genres_News,genres_Reality-TV,genres_Romance,genres_Sci-Fi,genres_Short,genres_Sport,genres_TV Movie,genres_Talk-Show,genres_Thriller,genres_War,genres_Western,original_language,production_companies,production_countries,director,actor
0,1,0,0,1.0,0,1.273072,0.0,5,4.9,18.0,1550.0,5.6,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5,0,10,10


In [ ]:
data.describe()

,titleType,startYear,isAdult,runtime,budget,popularity,revenue,status,vote_average,vote_count,numVotes,averageRating,release_month,genres_Action,genres_Adult,genres_Adventure,genres_Animation,genres_Biography,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Film-Noir,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Musical,genres_Mystery,genres_News,genres_Reality-TV,genres_Romance,genres_Sci-Fi,genres_Short,genres_Sport,genres_TV Movie,genres_Talk-Show,genres_Thriller,genres_War,genres_Western,original_language,production_companies,production_countries,director,actor
count,45307.000000,45307.000000,45307.000000,45307.000000,4.530700e+04,45307.000000,4.530700e+04,45307.000000,45307.000000,45307.000000,4.530700e+04,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000,45307.000000
mean,0.595979,0.808462,0.000596,98.097115,4.235149e+06,2.927732,1.124325e+07,4.995453,5.625003,110.222151,1.613350e+04,6.322242,6.468559,0.142230,0.000464,0.101287,0.045799,0.051052,0.325910,0.135321,0.099344,0.501733,0.054804,0.053435,0.010616,0.000044,0.037875,0.100051,0.033107,0.020416,0.061933,0.002273,0.000066,0.152934,0.050323,0.022712,0.017128,0.000022,0.000088,0.117134,0.027854,0.018584,1.385614,4.713466,2.792416,9.933167,9.922771
std,1.746451,0.845630,0.024405,35.340389,1.745001e+07,6.013227,6.443678e+07,0.174847,1.914672,492.101919,6.829993e+04,1.144720,3.631098,0.349289,0.021524,0.301711,0.209050,0.220106,0.468719,0.342070,0.299127,0.500003,0.227600,0.224903,0.102489,0.006644,0.190896,0.300071,0.178919,0.141421,0.241037,0.047626,0.008137,0.359928,0.218614,0.148985,0.129748,0.004698,0.009396,0.321584,0.164557,0.135053,2.634037,1.002802,2.724349,0.696581,0.759826
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,5.000000e+00,1.100000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,87.000000,0.000000e+00,0.388835,0.000000e+00,5.000000,5.000000,3.000000,5.050000e+02,5.700000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,10.000000,10.000000
50%,0.000000,1.000000,0.000000,96.000000,0.000000e+00,1.130550,0.000000e+00,5.000000,6.000000,10.000000,1.470000e+03,6.500000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,10.000000,10.000000
75%,0.000000,2.000000,0.000000,108.000000,0.000000e+00,3.694370,0.000000e+00,5.000000,6.800000,34.000000,5.356000e+03,7.100000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,6.000000,10.000000,10.000000
max,8.000000,2.000000,1.000000,1256.000000,3.800000e+08,547.488298,2.787965e+09,6.000000,10.0

In [ ]:
data.to_csv('processed_data.csv',index = False)

In [ ]:
data = pd.read_csv('processed_data.csv')
overview = pd.read_csv('overview_bert_features.csv',index_col=0)
review = pd.read_csv('sentiment_score.csv',index_col=0)

In [ ]:
data['id'] = df['id']
data['primaryTitle'] = df['primaryTitle']

data.set_index('id',inplace = True)
overview.set_index('id',inplace = True)
data = pd.concat([data,overview.iloc[:,:-1]],axis = 1)

data.set_index('primaryTitle',inplace = True)
review.set_index('primaryTitle',inplace = True)
data['sentiment_score'] = review['sentiment_score']
data.reset_index(inplace = True,drop=True)

data.dropna(inplace = True)
data.shape

(37200, 816)

In [ ]:
data.head(1)

,titleType,startYear,isAdult,runtime,budget,popularity,revenue,status,vote_average,vote_count,numVotes,averageRating,release_month,genres_Action,genres_Adult,genres_Adventure,genres_Animation,genres_Biography,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Film-Noir,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Musical,genres_Mystery,genres_News,genres_Reality-TV,genres_Romance,genres_Sci-Fi,genres_Short,genres_Sport,genres_TV Movie,genres_Talk-Show,genres_Thriller,genres_War,genres_Western,original_language,production_companies,production_countries,director,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,sentiment_score
0,1,0,0,1.0,0,1.273072,0.0,5,4.9,18.0,1550.0,5.6,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5,0,10,10,-0.212757,-0.119363,-0.467321,-0.104196,0.156821,0.016843,0.453776,0.254775,-0.587311,-0.117029,0.024697,-0.210755,-0.025135,0.439666,-0.323777,0.046454,-0.238279,0.278896,0.328746,-0.284318,-0.250684,-0.184989,-0.017924,-0.068809,-0.277619,-0.203015,0.024292,0.126

In [ ]:
data = data.drop(['popularity','vote_average','vote_count','numVotes'],axis = 1)

In [ ]:
data.to_csv('processed_data_for_lime.csv')